# emoji 크롤링 및 파싱

In [87]:
import pandas as pd

In [1]:
import xml.etree.ElementTree as ET
tree = ET.parse('country_data.xml')
root = tree.getroot()

FileNotFoundError: [Errno 2] No such file or directory: 'country_data.xml'

In [3]:
from urllib.request import urlopen
import xml.etree.ElementTree as ET
with urlopen('https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/ko.xml') as f:
    tree = ET.parse(f)
    root = tree.getroot()
    
    root

In [4]:
root.text

'\n\t'

In [9]:
for i in root:
    print(i.tag, i.attrib)

identity {}
annotations {}


In [10]:
print(root[1][1].text)

하얀 피부


In [11]:
print(root)

<Element 'ldml' at 0x7fed9922a7c0>


In [13]:
type(root)

xml.etree.ElementTree.Element

In [14]:
len(root)

2

In [15]:
root[1]

<Element 'annotations' at 0x7fed9922a900>

In [26]:
root[1][2].text, root[0][2].text

IndexError: child index out of range

In [36]:
root[1][2].text

'연한 갈색 피부 | 타입 삼 | 피부톤'

In [53]:
root[1][12].attrib, root[1][12].text

({'cp': '🤌'}, '꼬집는 | 빈정대는 | 손가락 | 손동작 | 질문')

In [54]:
root[1][13].attrib, root[1][13].text

({'cp': '🤌', 'type': 'tts'}, '꼬집는 손가락')

In [55]:
root[1][0].attrib, root[1][0].text

({'cp': '🏻'}, '타입 일 다시 이 | 피부톤 | 하얀 피부')

In [79]:
root.findall(".//*[@type='tts']")[13].attrib['cp'], root.findall(".//*[@type='tts']")[13].text

('🦬', '들소')

In [76]:
len(root.findall(".//*[@type='tts']"))

1497

In [81]:
type(root.findall(".//*[@type='tts']")[13].attrib['cp'])

str

In [82]:
root.findall(".//*[@type='tts']").attrib['cp']

AttributeError: 'list' object has no attribute 'attrib'

In [84]:
a=map(lambda x: x.attrib['cp'], root.findall(".//*[@type='tts']")) 

TypeError: 'map' object is not subscriptable

In [96]:
from urllib.request import urlopen
import xml.etree.ElementTree as ET
with urlopen('https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/ko.xml') as f:
    df = pd.read_xml(f, xpath=".//annotation")

In [97]:
df

,cp,annotation,type
0,🏻,타입 일 다시 이 | 피부톤 | 하얀 피부,None
1,🏻,하얀 피부,tts
2,🏼,연한 갈색 피부 | 타입 삼 | 피부톤,None
3,🏼,연한 갈색 피부,tts
4,🏽,갈색 피부 | 타입 사 | 피부톤,None
...,...,...,...
2989,🏳‍⚧,트랜스젠더 깃발,tts
2990,🏳‍🌈,깃발 | 무지개,None
2991,🏳‍🌈,무지개 깃발,tts
2992,🏴‍☠,보물섬 | 약탈자 | 해골 깃발 | 해적 | 해적 깃발,None


In [105]:
df[df['type']=='tts'].reset_index(drop=True)

,cp,annotation,type
0,🏻,하얀 피부,tts
1,🏼,연한 갈색 피부,tts
2,🏽,갈색 피부,tts
3,🏾,진한 갈색 피부,tts
4,🏿,검은색 피부,tts
...,...,...,...
1492,🏴,펄럭이는 검은색 깃발,tts
1493,🏳,펄럭이는 흰색 깃발,tts
1494,🏳‍⚧,트랜스젠더 깃발,tts
1495,🏳‍🌈,무지개 깃발,tts


## 사용법
### 1. unicode 이모지 오픈소스 xml 크롤링
- emoji unicode 컨소시움 (https://github.com/unicode-org)에서 cldr 페이지로 접근한다.
- 언어별로 xml로 정리되어있는 원본페이지를 찾고, xml import를 한다.
- ".//annotation"위치에 있는 모든 데이터를 변환한다.
    - 도큐먼트 설명
        - //annotation[@cp] : emoji (key)
        - //annotation : name (value)
        - //annotation[@type] : TTS타입인지 아닌지 (condition)
- type이 tts인 도큐먼트의 emoji(key) : name(value) 쌍을 구한다.
### 2. 1번의 방법 * 한/영/프/스
1. 한: https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/ko.xml
1. 영: https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/en.xml
1. 프: https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/fr.xml
1. 스: https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/es.xml

### 3. csv export
- 한/영/프/스 도큐먼트를 한 파일로 통합하여 csv export
- csv 설명
    - emoji: emoji
    - tts_ko: 한국어 name
    - tts_en: 영어 name
    - tts_fr: 불어 name
    - tts_es: 스페인 name

# 코드
## 1. input 필드, 예시 데이터 추출

In [135]:
# 한국어 -> 영어 -> 불어 -> 스페인 순서
urls=["https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/ko.xml",
      "https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/en.xml",
      "https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/fr.xml",
      "https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/es.xml"]
cols=['ko','en','fr','es']


import pandas as pd
from urllib.request import urlopen
import xml.etree.ElementTree as ET
with urlopen('https://raw.githubusercontent.com/unicode-org/cldr/latest/common/annotations/ko.xml') as f:
    df = pd.read_xml(f, xpath=".//annotation")
df[df['type']=='tts'].reset_index(drop=True)

,cp,annotation,type
0,🏻,하얀 피부,tts
1,🏼,연한 갈색 피부,tts
2,🏽,갈색 피부,tts
3,🏾,진한 갈색 피부,tts
4,🏿,검은색 피부,tts
...,...,...,...
1492,🏴,펄럭이는 검은색 깃발,tts
1493,🏳,펄럭이는 흰색 깃발,tts
1494,🏳‍⚧,트랜스젠더 깃발,tts
1495,🏳‍🌈,무지개 깃발,tts


## 2. 언어별로 테이블 추출 및 concat
1. xml to DataFrame 변환
1. //annotation 도큐먼트만 추출, type=='tts'인 것들만
1. 언어별로 추출한 것을 concat
1. concat할때 cp열과 annotation열 (name) 합침

In [203]:
# concat 리스트 선언
concat_df=[]

# lang별로 도큐먼트 추출 및 concat
for i in range(len(urls)):
    with urlopen(urls[i]) as f:
        df = pd.read_xml(f, xpath=".//annotation")
    temp=df[df['type']=='tts'].reset_index(drop=True)
    #temp.rename(columns={'annotation':cols[i]}, inplace=True)
    
    # cp열, annotation열만 추출하여 리스트에 담기
    concat_df.append(temp[['cp','annotation']])

In [211]:
# concat 리스트의 데이터프레임들을 concat 실시
test=pd.concat(concat_df, axis=1, join='inner')
test=test.loc[:,~test.T.duplicated()]

# 언어별로 열 이름 지정
test.columns=['cp']+cols

# 결과 출력
test[:10]

,cp,ko,en,fr,es
0,🏻,하얀 피부,light skin tone,peau claire,tono de piel claro
1,🏼,연한 갈색 피부,medium-light skin tone,peau moyennement claire,tono de piel claro medio
2,🏽,갈색 피부,medium skin tone,peau légèrement mate,tono de piel medio
3,🏾,진한 갈색 피부,medium-dark skin tone,peau mate,tono de piel oscuro medio
4,🏿,검은색 피부,dark skin tone,peau foncée,tono de piel oscuro
5,⚧,트랜스젠더 기호,transgender symbol,symbole de la communauté transgenre,símbolo de transgénero
6,🤌,꼬집는 손가락,pinched fingers,bout des doigts joints,dedos juntos apuntando hacia arriba
7,🥲,눈물 흘리며 웃는 얼굴,smiling face with tear,Visage souriant avec une larme,cara sonriente con lágrima
8,🥷,닌자,ninja,ninja,ninja
9,🥸,변장한 얼굴,disguised face,visage déguisé,cara disfrazada


In [212]:
test.to_csv('1.csv')